# My first auto Correct system 

In this notebook , we  will implement an auto-correct system !! To correct words that are 1 and 2 edit distances away , we will see later what does it mean ...


## Outline
- [0. Overview](#0)
   
- [1. Data Preprocessing](#1)
    
- [2. String Manipulation](#2)
    
- [3. Combining the edits](#3)
    
- [4. Minimum Edit Distance](#4)
    
- [5. Backtrace ](#5)

<a name='0'></a>
## 0. Overview

We use autocorrect every day on our cell phone and computer. 
Through this program , we will explore what really goes on behind the scenes. Of course, the model we are about to implement is not identical to the one used in your phone, but it is still quite good. 

Auto Correct systems are used everywhere. 
- For example, if you type in the word **"I am hapy"**, chances are very high that you meant to write **"I am happy"**. 

<a name='0-1'></a>
#### 0.1 Edit Distance

As i said before , we will implement models that correct words that are 1 and 2 edit distances away. 
- We say two words are n edit distance away from each other when we need n edits to change one word into another. 

An edit could consist of one of the following options: 

- Delete (remove a letter): ‘hat’ => ‘at, ha, ht’
- Switch (swap 2 adjacent letters): ‘eta’ => ‘eat, tea,...’
- Replace (change 1 letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
- Insert (add a letter): ‘te’ => ‘the, ten, ate, ...’


This auto-correct  was first created by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) in 2007. 


The goal is to compute the following probability:

$$P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} $$

This equation is known as  [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem). 
- It's says that the probability of a word being correct $P(c|w) $is equal to the probability of having a certain word $w$, given that it is correct $P(w|c)$, multiplied by the probability of being correct in general $P(c)$ divided by the probability of that word $w$ appearing $P(w)$ in general.


<a name='1'></a>
# Part 1: Data Preprocessing 

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

Given a file X whcih contains our vocabulary , we will read it then transform it's content to lowercase

In [2]:
# return a list of words from a file 
def process_data(file_name):
    
    words = [] 
    str_total=''

    with open(file_name, 'r') as current:
        if len(current.read()) == 0:
            print('FILE IS EMPTY')
        else:
            current.seek(0)
            for line in current.readlines():
                str_total+= line.lower()
    words=re.findall('\w+', str_total)
    
    return words

In [4]:
word_l = process_data('shakespeare.txt')
vocab = set(word_l) #To eliminate duplicates words 
print(f"There are {len(vocab)} unique words in the vocabulary.")
print(f"The first five words in the text are: \n{word_l[0:5]}")

There are 6116 unique words in the vocabulary.
The first five words in the text are: 
['o', 'for', 'a', 'muse', 'of']


Now , we eill Implement a `get_count` function that returns a dictionary
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus. 

For example, given the following sentence: **"I am happy because I am learning"**, your dictionary should return the following: 
<table style="width:20%">

  <tr>
    <td> <b>Key </b>  </td>
    <td> <b>Value </b> </td> 


  </tr>
  <tr>
    <td> I  </td>
    <td> 2</td> 
 
  </tr>
   
  <tr>
    <td>am</td>
    <td>2</td> 
  </tr>

  <tr>
    <td>happy</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>because</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>learning</td>
    <td>1</td> 
  </tr>
</table>


In [5]:
def get_count(word_l):
    
    word_count_dict = {}  
    word_count_dict = Counter(word_l)
           
    return word_count_dict

In [6]:
word_count_dict = get_count(word_l)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'world' is {word_count_dict.get('thee',0)}")

There are 6116 key values pairs
The count for the word 'world' is 240


Given the dictionary of word counts, we will compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M} $$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7} $$


In [7]:
def get_probs(word_count_dict):
   
    probs = {}  # return this variable correctly
    

    for word in word_count_dict.keys():
        probs[word]=word_count_dict[word]/sum(word_count_dict.values())
    
    return probs

In [8]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('world') is {probs['thee']:.4f}")

Length of probs is 6116
P('world') is 0.0045


<a name='2'></a>
# Part 2: String Manipulations

Now, that we have computed $P(w_i)$ for all the words in the corpus, we will write a few functions to manipulate strings with  four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


In [9]:
def delete_letter(word, verbose=False):
    
    delete_l = []
    split_l = []
    
    split_l=[(word[:i],word[i:]) for i in range (len(word))]
    #delete_l=[word.replace(word[i],'') for i in range (len(word))] # Without comprehensive List
    delete_l=[L+R[1:] for L,R in split_l if R] #With comprehensive List 
            

    if verbose: print(f"input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [10]:
delete_word_l = delete_letter(word="world",
                        verbose=True)

input word world, 
split_l = [('', 'world'), ('w', 'orld'), ('wo', 'rld'), ('wor', 'ld'), ('worl', 'd')], 
delete_l = ['orld', 'wrld', 'wold', 'word', 'worl']


In [11]:
def switch_letter(word, verbose=False):
        
    switch_l = []
    split_l = []
      
    split_l=[(word[:i],word[i:]) for i in range (len(word))]
    switch_l=[ L+R[1]+R[0]+R[2:] for L,R in split_l if len(R)>=2]
   
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [13]:
switch_word_l = switch_letter(word="sad",
                         verbose=True)

Input word = sad 
split_l = [('', 'sad'), ('s', 'ad'), ('sa', 'd')] 
switch_l = ['asd', 'sda']


In [14]:
def replace_letter(word, verbose=False):
    
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    replace_l = []
    split_l = []
    
    
    split_l=[(word[:i],word[i:]) for i in range (len(word))]
    for L,R in  split_l:
        for c in letters:
            if c!=R[0]:
                replace_l.append(L+c+R[1:])
                
    replace_set=set(replace_l)
  
    # turn the set back into a list and sort it, for easier viewing
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [15]:
replace_l = replace_letter(word='tea',
                              verbose=True)


Input word = tea 
split_l = [('', 'tea'), ('t', 'ea'), ('te', 'a')] 
replace_l ['aea', 'bea', 'cea', 'dea', 'eea', 'fea', 'gea', 'hea', 'iea', 'jea', 'kea', 'lea', 'mea', 'nea', 'oea', 'pea', 'qea', 'rea', 'sea', 'taa', 'tba', 'tca', 'tda', 'teb', 'tec', 'ted', 'tee', 'tef', 'teg', 'teh', 'tei', 'tej', 'tek', 'tel', 'tem', 'ten', 'teo', 'tep', 'teq', 'ter', 'tes', 'tet', 'teu', 'tev', 'tew', 'tex', 'tey', 'tez', 'tfa', 'tga', 'tha', 'tia', 'tja', 'tka', 'tla', 'tma', 'tna', 'toa', 'tpa', 'tqa', 'tra', 'tsa', 'tta', 'tua', 'tva', 'twa', 'txa', 'tya', 'tza', 'uea', 'vea', 'wea', 'xea', 'yea', 'zea']


In [16]:
def insert_letter(word, verbose=False):

    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    insert_l = []
    split_l = []
    
    split_l=[(word[:i],word[i:]) for i in range (len(word)+1)]
    insert_l=[L+c+R for L,R in  split_l for c in letters]

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [18]:
insert_l = insert_letter('in', True)
print(f"Number of strings output by insert_letter('at') is {len(insert_l)}")

Input word in 
split_l = [('', 'in'), ('i', 'n'), ('in', '')] 
insert_l = ['ain', 'bin', 'cin', 'din', 'ein', 'fin', 'gin', 'hin', 'iin', 'jin', 'kin', 'lin', 'min', 'nin', 'oin', 'pin', 'qin', 'rin', 'sin', 'tin', 'uin', 'vin', 'win', 'xin', 'yin', 'zin', 'ian', 'ibn', 'icn', 'idn', 'ien', 'ifn', 'ign', 'ihn', 'iin', 'ijn', 'ikn', 'iln', 'imn', 'inn', 'ion', 'ipn', 'iqn', 'irn', 'isn', 'itn', 'iun', 'ivn', 'iwn', 'ixn', 'iyn', 'izn', 'ina', 'inb', 'inc', 'ind', 'ine', 'inf', 'ing', 'inh', 'ini', 'inj', 'ink', 'inl', 'inm', 'inn', 'ino', 'inp', 'inq', 'inr', 'ins', 'int', 'inu', 'inv', 'inw', 'inx', 'iny', 'inz']
Number of strings output by insert_letter('at') is 78


<a name='3'></a>

# Part 3: Combining the edits

Now that we have implemented the string manipulations, we will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

<a name='3-1'></a>
## 3.1 Edit one letter

`edit_one_letter` : To get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation. 

In [19]:
def edit_one_letter(word, allow_switches = True):
    
    edit_one_set = set()
    
    edit_one_set=set(delete_letter(word, verbose=False)+
                     switch_letter(word, verbose=False)+
                     replace_letter(word, verbose=False)+
                     insert_letter(word, verbose=False)                     
                    )

    return edit_one_set

In [20]:
tmp_word = "on"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"Number of outputs from edit_one_letter('at') is {len(edit_one_letter('at'))}")

input word on 
edit_one_l 
['an', 'aon', 'bn', 'bon', 'cn', 'con', 'dn', 'don', 'en', 'eon', 'fn', 'fon', 'gn', 'gon', 'hn', 'hon', 'in', 'ion', 'jn', 'jon', 'kn', 'kon', 'ln', 'lon', 'mn', 'mon', 'n', 'nn', 'no', 'non', 'o', 'oa', 'oan', 'ob', 'obn', 'oc', 'ocn', 'od', 'odn', 'oe', 'oen', 'of', 'ofn', 'og', 'ogn', 'oh', 'ohn', 'oi', 'oin', 'oj', 'ojn', 'ok', 'okn', 'ol', 'oln', 'om', 'omn', 'ona', 'onb', 'onc', 'ond', 'one', 'onf', 'ong', 'onh', 'oni', 'onj', 'onk', 'onl', 'onm', 'onn', 'ono', 'onp', 'onq', 'onr', 'ons', 'ont', 'onu', 'onv', 'onw', 'onx', 'ony', 'onz', 'oo', 'oon', 'op', 'opn', 'oq', 'oqn', 'or', 'orn', 'os', 'osn', 'ot', 'otn', 'ou', 'oun', 'ov', 'ovn', 'ow', 'own', 'ox', 'oxn', 'oy', 'oyn', 'oz', 'ozn', 'pn', 'pon', 'qn', 'qon', 'rn', 'ron', 'sn', 'son', 'tn', 'ton', 'un', 'uon', 'vn', 'von', 'wn', 'won', 'xn', 'xon', 'yn', 'yon', 'zn', 'zon']

Number of outputs from edit_one_letter('at') is 129


<a name='3-2'></a>
## Part 3.2 Edit two letters

`edit_two_letters` : To return a set of words that are two edits away. 
Note that creating additional edits based on the `edit_one_letter` function may 'restore' some one_edits to zero or one edits. That is allowed here.

In [21]:
def edit_two_letters(word, allow_switches = True):
       
    edit_two_set = set()
    

    tmp_edit_one_set=edit_one_letter(word, allow_switches = True)
    edit_one_set=sorted(list(tmp_edit_one_set))
    for word_i in edit_one_set:
        l1=edit_one_letter(word_i, allow_switches = True)
        for i in l1:
            edit_two_set.add(i)
            
    
    return edit_two_set

In [22]:
tmp_edit_two_set = edit_two_letters("a")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"Number of strings that are 2 edit distances from 'at' is {len(edit_two_letters('at'))}")

Number of strings with edit distance of two: 2654
First 10 strings ['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag']
Last 10 strings ['zv', 'zva', 'zw', 'zwa', 'zx', 'zxa', 'zy', 'zya', 'zz', 'zza']
Number of strings that are 2 edit distances from 'at' is 7154


<a name='3-3'></a>
## Part 3-3: suggest spelling suggestions

Now you will use our  `edit_two_letters` function to get a set of all the possible 2 edits on your word. We will then use those strings to get the most probable word we meant to type aka your typing suggestion.


`get_corrections`:  returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word). 


Note: 
- Edits of one or two letters may 'restore' strings to either zero or one edit. This algorithm accounts for this by preferentially selecting lower distance edits first.

In [23]:

def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''
    
    suggestions = []
    n_best = []
    best_words={}
    
    if word in vocab :
        suggestions.append(word)
    else:
        l1=edit_one_letter(word, allow_switches = True)
        l2=edit_two_letters(word, allow_switches = True)
        suggestions=(l1.intersection(vocab)) or (l2.intersection(vocab)) or word
   
    for k in suggestions:
        if k in vocab :
            best_words[k]=probs[k] 
        else :
            best_words[k]=0
            
    n_best= Counter(best_words).most_common(n)  
           
    if verbose: print("suggestions = ", suggestions)

    return n_best

In [26]:
my_word = 'hapy' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


suggestions =  {'happy', 'hap', 'haply'}
word 0: happy, probability 0.000317
word 1: haply, probability 0.000131


In [28]:
my_word = 'hrs' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


suggestions =  {'has', 'his', 'rs', 'hers'}
word 0: his, probability 0.008095
word 1: has, probability 0.000578


In [32]:
my_word = 'kannot' 
tmp_corrections = get_corrections(my_word, probs, vocab, 2, verbose=True)
for i, word_prob in enumerate(tmp_corrections):
    print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")


suggestions =  {'cannot'}
word 0: cannot, probability 0.000951


<a name='4'></a>
# Part 4: Minimum Edit distance

Now that we have implemented our auto-correct sys , how we can evaluate the similarity between two strings? For example: 'waht' and 'what'

Also how do we efficiently find the shortest path to go from the word, 'waht' to the word 'what'?

In this part , We will implement a dynamic programming system that will tell us the minimum number of edits required to convert a string into another string.

<a name='4-1'></a>
### Part 4.1 Dynamic Programming

Dynamic Programming breaks a problem down into subproblems which can be combined to form the final solution. Here, given a string source[0..i] and a string target[0..j], we will compute all the combinations of substrings[i, j] and calculate their edit distance. To do this efficiently, we will use a table to maintain the previously computed substrings and use those to calculate larger substrings.

We have to create a matrix and update each element in the matrix as follows:  

$$\text{Initialization}$$

\begin{align}
D[0,0] &= 0 \\
D[i,0] &= D[i-1,0] + del\_cost(source[i]) \\
D[0,j] &= D[0,j-1] + ins\_cost(target[j]) \\
\end{align}


$$\text{Per Cell Operations}$$
\begin{align}
 \\
D[i,j] =min
\begin{cases}
D[i-1,j] + del\_cost\\
D[i,j-1] + ins\_cost\\
D[i-1,j-1] + \left\{\begin{matrix}
rep\_cost; & if src[i]\neq tar[j]\\
0 ; & if src[i]=tar[j]
\end{matrix}\right.
\end{cases}
\
\end{align}

So converting the source word **play** to the target word **stay**, using an input cost of 1, a delete cost of 1, and replace cost of 2 would give you the following table:
<table style="width:20%">

  <tr>
    <td> <b> </b>  </td>
    <td> <b># </b>  </td>
    <td> <b>s </b>  </td>
    <td> <b>t </b> </td> 
    <td> <b>a </b> </td> 
    <td> <b>y </b> </td> 
  </tr>
   <tr>
    <td> <b>  #  </b></td>
    <td> 0</td> 
    <td> 1</td> 
    <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
 
  </tr>
  <tr>
    <td> <b>  p  </b></td>
    <td> 1</td> 
 <td> 2</td> 
    <td> 3</td> 
    <td> 4</td> 
   <td> 5</td>
  </tr>
   
  <tr>
    <td> <b> l </b></td>
    <td>2</td> 
    <td>3</td> 
    <td>4</td> 
    <td>5</td> 
    <td>6</td>
  </tr>

  <tr>
    <td> <b> a </b></td>
    <td>3</td> 
     <td>4</td> 
     <td>5</td> 
     <td>4</td>
     <td>5</td> 
  </tr>
  
   <tr>
    <td> <b> y </b></td>
    <td>4</td> 
      <td>5</td> 
     <td>6</td> 
     <td>5</td>
     <td>4</td> 
  </tr>
  

</table>



The operations used in this algorithm are 'insert', 'delete', and 'replace', 'switch'is not used here.

In [33]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 

    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): # Replace None with the proper range
        D[row,0] = D[row-1,0]+del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): # Replace None with the proper range
        D[0,col] = D[0,col-1]+ins_cost 
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row-1]==target[col-1]:
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min(D[row-1,col-1]+r_cost,D[row,col-1]+ins_cost,D[row-1,col]+del_cost)

    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m,n]
    
    return D, med

In [34]:
# testing implementation 
source =  'play'
target = 'stay'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  s  t  a  y
#  0  1  2  3  4
p  1  2  3  4  5
l  2  3  4  5  6
a  3  4  5  4  5
y  4  5  6  5  4


In [35]:
# testing your implementation 
source =  'eer'
target = 'near'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  3 

   #  n  e  a  r
#  0  1  2  3  4
e  1  2  1  2  3
e  2  3  2  3  4
r  3  4  3  4  3


We can now test several of our routines at once:

In [37]:
source = "in"
targets = edit_one_letter(source,allow_switches = False)  #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 1: print(source, t, min_edits)

in ni 2


The 'replace()' routine utilizes all letters a-z one of which returns the original word.

In [38]:
source = "in"
targets = edit_two_letters(source,allow_switches = False) #disable switches since min_edit_distance does not include them
for t in targets:
    _, min_edits = min_edit_distance(source, t,1,1,1)  # set ins, del, sub costs all to one
    if min_edits != 2 and min_edits != 1: print(source, t, min_edits)

in nei 3
in npi 3
in nmi 3
in nvi 3
in nhi 3
in nqi 3
in nai 3
in nwi 3
in nki 3
in nji 3
in nci 3
in nui 3
in nri 3
in ngi 3
in nzi 3
in nyi 3
in noi 3
in nbi 3
in nti 3
in nli 3
in nxi 3
in nfi 3
in in 0
in ndi 3
in nsi 3


<a name='5'></a>

# Part 5:  Backtrace


To be added later